## API Function Definitions

In [6]:
import pandas as pd
from datetime import datetime, timedelta
import requests
import json

import certifi
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [8]:
#Input values
fleet_size = [1000,10000,">30000"]
mean_dvmt = [45,35,25]
temp_c = [40,30,20,10,0,-10,-20]
pev_type = ['PHEV20','PHEV50','BEV100','BEV250']
pev_dist = ['EQUAL_DIST','PHEV_DOM','BEV_DOM']
vehicle_class_dist = ['Sed20_Suv80','Sed50_Suv50','Sed80_Suv20']
day_of_week = ['weekday','weekend']
home_access = ['HA50','HA75','HA100']
home_power_dist = ["L2"]
work_power_dist = ['L2_20','L2_50','L2_100']
pref_dist = ['Home60','Home80','Home100']
dest_type = ['Home','Public','Work']
dest_chg_level = ['L1','L2','L3']


In [13]:
def selection_API_Run(fleet_size,mean_dvmt,temp_c,pev_type,pev_dist,vehicle_class_dist,home_access,work_power_dist,pref_dist,dest_chg_level,day_of_week,dest_type,test=1):    
    params=[]
    #Don't need home_power_dist since it only has one choice
    #day_of_week and dest_type are used to generate plots- therefore all selections are used (and they are omitted from choice selection)
    
    for var_name,var in zip(["fleet_size","mean_dvmt","temp_c","pev_type","pev_dist","vehicle_class_dist","home_access","work_power_dist","pref_dist"],[fleet_size,mean_dvmt,temp_c,pev_type,pev_dist,vehicle_class_dist,home_access,work_power_dist,pref_dist]):
        if test!=1:
            print("Please select a value for "+var_name+".")
            for i in range(0,len(var)):
                print(str(i)+") "+str(var[i]))
            selection = input("Enter an integer selection from above (Ex. 0, 1, 2..): ") #User selects parameter choice from list
            while len(selection)!=1:
                selection = input("Enter just the number corresponding to a choice from above (Ex. 0, 1, 2..): ")
            selection = int(selection)
            print(var[selection])
            params.append(var[selection])         #Will be a list of parameters in the order listed in the for loop
            print("\n")
        else:
            print(var[0])
            params.append(var[0])

    #Generate load profiles for home, public, and work on both weekends and weekdays and for different charger levels according to the selected parameters
    record_json={}
    for dow in day_of_week:
        record_json[dow]={}
        for location in dest_type:
            record_json[dow][location]={}
            for chg_level in dest_chg_level:
                print(dow,location, chg_level)
                if (location=="Home" and chg_level!="L3") or (location=="Public" and chg_level!="L1"): #Skip home level 3 or public level 1
                    url = """https://evi-pro-lite-v0.afdc-stage.nrel.gov/evi-pro-lite/api/v1/load-profile?fleet_size=%s&mean_dvmt=%s&temp_c=%s&pev_type=%s&pev_dist=%s&vehicle_class_dist=%s&home_access=%s&home_power_dist=L2&work_power_dist=%s&pref_dist=%s&dest_chg_level=%s&day_of_week=%s&dest_type=%s""" %(params[0],params[1],params[2],params[3],params[4],params[5],params[6],params[7],params[8],chg_level,dow,location)
                    url=url.replace("\\", "")
                    record_str = requests.get(url).text
                    record_str = record_str.replace("'", "\"")
                    print(record_str)
                    raw_json=json.loads(record_str)
                    record_json[dow][location][chg_level] = [float(i) for i in raw_json['load_profile']]
                else:
                    pass
    return record_json

In [14]:
#Run user selection
result = selection_API_Run(fleet_size,mean_dvmt,temp_c,pev_type,pev_dist,vehicle_class_dist,home_access,work_power_dist,pref_dist,dest_chg_level,day_of_week,dest_type)

1000
45
40
PHEV20
EQUAL_DIST
Sed20_Suv80
HA50
L2_20
Home60
weekday Home L1
{"metadata":{"version":"0.0.0","inputs":{"fleet_size":"1000","mean_dvmt":"45","temp_c":"40","pev_type":"PHEV20","pev_dist":"EQUAL_DIST","vehicle_class_dist":"Sed20_Suv80","home_access":"HA50","home_power_dist":"L2","work_power_dist":"L2_20","pref_dist":"Home60","dest_chg_level":"L1","day_of_week":"weekday","dest_type":"Home"}},"load_profile":[19.6,19.6,19.6,19.51,14.76,11.2,11.2,7.57,5.6,3.18,3.99,2.8,2.8,2.8,2.8,2.8,2.8,2.8,2.8,2.8,2.8,2.8,0.0,0.22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.68,2.8,0.0,2.8,2.8,2.8,2.8,2.8,3.55,5.6,5.6,7.47,9.96,11.2,11.2,8.4,8.4,8.4,11.2,11.08,8.4,8.4,13.63,14.0,16.8,15.26,14.93,16.8,19.6,19.6,18.46,17.73,25.2,27.81,28.0,28.0,28.0,30.8,36.4,38.27,42.06,39.2,39.2,40.69,39.35,39.2,33.79,39.02,47.6,46.63,46.02,42.0,41.26,41.07,40.61,39.2,37.07,34.11,32.49,30.16,25.22,24.92,22.4,19.94]}
weekday Home L2
{"metadata":{"version":"0.0.0","inputs":{"fleet_size":"1000","mean_dvmt":"45","temp_c":"40

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

## Plotting

In [ ]:
fig = plt.figure(figsize = (12,5))
ax = plt.axes()
xaxis_labels = list(range(0,96))

for i in result.keys():   
    ax.plot(xaxis_labels,result[i]['load_profile'],linewidth=3)


plt.legend(result.keys(),fontsize = 14)
plt.xlabel('Time of Day',size=18)
plt.ylabel('Grid Load [kW]',size=18)
plt.title('Grid load with varying fleet compositions',size=18)
plt.xticks(size=14)
plt.yticks(size=14)

ymin, ymax = ax.get_ylim()
custom_ticks = np.linspace(ymin, ymax, 10, dtype=int)
ax.set_yticks(custom_ticks)
ax.set_yticklabels(custom_ticks)

for i in result.keys(): 
    print(i,':\n',result[i]['metadata']['inputs'])

In [ ]:
fig = plt.figure(figsize = (12,5))
ax = plt.axes()
xaxis_labels = list(range(0,96))

for i in result2.keys():   
    ax.plot(xaxis_labels,result2[i]['load_profile'],linewidth=3)

plt.legend(result2.keys(),fontsize = 14)
plt.xlabel('Time of Day',size=18)
plt.ylabel('Grid Load [kW]',size=18)
plt.title('Grid load with varying charger location preferences',size=18)
plt.xticks(size=14)
plt.yticks(size=14)

ymin, ymax = ax.get_ylim()
custom_ticks = np.linspace(ymin, ymax, 10, dtype=int)
ax.set_yticks(custom_ticks)
ax.set_yticklabels(custom_ticks)

for i in result2.keys(): 
    print(i,':\n',result2[i]['metadata']['inputs'])

In [ ]:
print(result['small_PHEV_fleet']['metadata']['inputs'])

In [ ]:
result['small_PHEV_fleet']['load_profile'] = [float(i) for i in result['small_PHEV_fleet']['load_profile']]

plt.plot(result['small_PHEV_fleet']['load_profile'])

In [ ]:
pd.DataFrame.from_dict(result2[i]['metadata']['inputs'])

In [ ]:
#Create scenario specification DF
df = pd.DataFrame.from_dict(result2[list(result2.keys())[0]]['metadata']['inputs'], orient='index').T
for i in list(result2.keys())[1:]:
    df = df.append(pd.DataFrame.from_dict(result2[i]['metadata']['inputs'], orient='index').T)
for i in list(result.keys()):
    df = df.append(pd.DataFrame.from_dict(result[i]['metadata']['inputs'], orient='index').T)
df['scenario']=(list(result2.keys())+list(result.keys()))

cols = list(df.columns)
cols = [cols[-1]] + cols[:-1]
df = df[cols]
df.reset_index(inplace=True,drop=True)
df

In [ ]:
df.to_csv('/Users/dweigl/Documents/Projects/Load Profiles/API_Scenarios.csv')

## Testing

In [ ]:
#Simple API run with input params
def API_run(fleet_size,mean_dvmt,temp_c,pev_type,pev_dist,vehicle_class_dist,day_of_week,home_access,home_power_dist,work_power_dist,pref_dist,dest_type):    
    url = """https://evi-pro-lite.afdc-stage.nrel.gov/evi_pro_lite/api/v1/load_profile?fleet_size=%s&mean_dvmt=%s&temp_c=%s&pev_type=%s&pev_dist=%s&vehicle_class_dist=%s&day_of_week=%s&home_access=%s&home_power_dist=%s&work_power_dist=%s&dest_chg_level=L2&pref_dist=%s&dest_type=%s""" %(fleet_size,mean_dvmt,temp_c,pev_type,pev_dist,vehicle_class_dist,day_of_week,home_access,home_power_dist,work_power_dist,pref_dist,dest_type)
    url=url.replace("\\", "")
    record_str = requests.get(url, verify=False).text
    record_str = record_str.replace("'", "\"")
    record_json=json.loads(record_str)
    record_json['load_profile'] = [float(i) for i in record_json['load_profile']]
    return record_json

In [ ]:
result={}
result['heavy_BEV'] = API_run(fleet_size[1],mean_dvmt[0],temp_c[-1],pev_type[3],pev_dist[2],vehicle_class_dist[0],day_of_week[0],home_access[0],home_power_dist[0],work_power_dist[2],pref_dist[0],dest_type[2])
result['light_BEV'] = API_run(fleet_size[1],mean_dvmt[0],temp_c[2],pev_type[2],pev_dist[2],vehicle_class_dist[2],day_of_week[0],home_access[2],home_power_dist[0],work_power_dist[0],pref_dist[2],dest_type[2])
result['mid_demand'] = API_run(fleet_size[1],mean_dvmt[0],temp_c[0],pev_type[2],pev_dist[2],vehicle_class_dist[1],day_of_week[0],home_access[1],home_power_dist[0],work_power_dist[1],pref_dist[1],dest_type[2])
result['small_PHEV_fleet'] = API_run(fleet_size[2],mean_dvmt[0],temp_c[2],pev_type[1],pev_dist[1],vehicle_class_dist[2],day_of_week[0],home_access[1],home_power_dist[0],work_power_dist[1],pref_dist[1],dest_type[2])

In [ ]:
result2={}
result2['heavy_home'] = API_run(fleet_size[1],mean_dvmt[0],temp_c[-1],pev_type[2],pev_dist[2],vehicle_class_dist[0],day_of_week[1],home_access[0],home_power_dist[0],work_power_dist[2],pref_dist[0],dest_type[0])
result2['light_home'] = API_run(fleet_size[1],mean_dvmt[0],temp_c[-1],pev_type[2],pev_dist[2],vehicle_class_dist[2],day_of_week[0],home_access[2],home_power_dist[0],work_power_dist[0],pref_dist[2],dest_type[1])
result2['heavy_work'] = API_run(fleet_size[1],mean_dvmt[0],temp_c[-1],pev_type[2],pev_dist[2],vehicle_class_dist[1],day_of_week[0],home_access[1],home_power_dist[0],work_power_dist[1],pref_dist[1],dest_type[2])
result2['light_work'] = API_run(fleet_size[1],mean_dvmt[0],temp_c[-1],pev_type[2],pev_dist[2],vehicle_class_dist[2],day_of_week[1],home_access[1],home_power_dist[0],work_power_dist[1],pref_dist[1],dest_type[1])